In [1]:
from langchain_core.tools import tool, InjectedToolArg
from typing import Annotated
import requests

In [2]:
# tool creation
@tool
def getCurrencyConversionFactor(base_currency: str, target_currency: str) -> float:
    """ Get a currency conversion factor from base_currency to target_currency."""
    
    url = f"https://v6.exchangerate-api.com/v6/ae37a026e013b0886b1b29c5/pair/{base_currency}/{target_currency}"
    
    response = requests.get(url)
    return response.json()

In [3]:
getCurrencyConversionFactor.invoke({'base_currency': 'USD', 'target_currency': 'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745971201,
 'time_last_update_utc': 'Wed, 30 Apr 2025 00:00:01 +0000',
 'time_next_update_unix': 1746057601,
 'time_next_update_utc': 'Thu, 01 May 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.2184}

In [4]:
@tool
def convertCurrency(amount: float, conversion_factor: float) -> float: # we have set the conversion_factor as an injected tool arg which means it will be passed as an argument to the function automatically, llm can't pass value to this argument from the function
    """ Convert an amount of money using a conversion factor."""
    
    return amount * conversion_factor

In [5]:
convertCurrency.invoke({'amount': 100, 'conversion_factor': 82.5})

8250.0

In [6]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage

In [7]:
llm = ChatCohere(model='command-r')

In [8]:
# tool binding
binded_llm = llm.bind_tools([getCurrencyConversionFactor, convertCurrency])

In [9]:
binded_llm.kwargs['tools']

[ToolV2(type='function', function=ToolV2Function(name='getCurrencyConversionFactor', description='Get a currency conversion factor from base_currency to target_currency.', parameters={'type': 'object', 'properties': {'base_currency': {'type': 'string', 'description': ''}, 'target_currency': {'type': 'string', 'description': ''}}, 'required': ['base_currency', 'target_currency']})),
 ToolV2(type='function', function=ToolV2Function(name='convertCurrency', description='Convert an amount of money using a conversion factor.', parameters={'type': 'object', 'properties': {'amount': {'type': 'number', 'description': ''}, 'conversion_factor': {'type': 'number', 'description': ''}}, 'required': ['amount', 'conversion_factor']}))]

In [30]:
messages = [HumanMessage("What is the conversion factor from eur to inr, and based on that can you convert 86 eur to INR?")]

In [31]:
res1 = binded_llm.invoke(messages)

In [32]:
res1.tool_calls

[{'name': 'getCurrencyConversionFactor',
  'args': {'base_currency': 'EUR', 'target_currency': 'INR'},
  'id': 'getCurrencyConversionFactor_272vhd9hntva',
  'type': 'tool_call'}]

In [33]:
messages.append(res1)
messages

[HumanMessage(content='What is the conversion factor from eur to inr, and based on that can you convert 86 eur to INR?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='First, I will find out the conversion factor from EUR to INR. Then, I will multiply this conversion factor by 86 to convert 86 EUR into INR.', additional_kwargs={'id': 'afcba0ba-afb5-44d2-b85b-b8de470fd61f', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'First, I will find out the conversion factor from EUR to INR. Then, I will multiply this conversion factor by 86 to convert 86 EUR into INR.', 'tool_calls': [{'id': 'getCurrencyConversionFactor_272vhd9hntva', 'type': 'function', 'function': {'name': 'getCurrencyConversionFactor', 'arguments': '{"base_currency":"EUR","target_currency":"INR"}'}}], 'token_count': {'input_tokens': 994.0, 'output_tokens': 90.0}}, response_metadata={'id': 'afcba0ba-afb5-44d2-b85b-b8de470fd61f', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'First, I will find out the conversion fac

In [34]:
getconversionrateResponse = getCurrencyConversionFactor.invoke(res1.tool_calls[0])
getconversionrateResponse

ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1745971201, "time_last_update_utc": "Wed, 30 Apr 2025 00:00:01 +0000", "time_next_update_unix": 1746057601, "time_next_update_utc": "Thu, 01 May 2025 00:00:01 +0000", "base_code": "EUR", "target_code": "INR", "conversion_rate": 96.993}', name='getCurrencyConversionFactor', tool_call_id='getCurrencyConversionFactor_272vhd9hntva')

In [35]:
messages.append(getconversionrateResponse)
messages

[HumanMessage(content='What is the conversion factor from eur to inr, and based on that can you convert 86 eur to INR?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='First, I will find out the conversion factor from EUR to INR. Then, I will multiply this conversion factor by 86 to convert 86 EUR into INR.', additional_kwargs={'id': 'afcba0ba-afb5-44d2-b85b-b8de470fd61f', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'First, I will find out the conversion factor from EUR to INR. Then, I will multiply this conversion factor by 86 to convert 86 EUR into INR.', 'tool_calls': [{'id': 'getCurrencyConversionFactor_272vhd9hntva', 'type': 'function', 'function': {'name': 'getCurrencyConversionFactor', 'arguments': '{"base_currency":"EUR","target_currency":"INR"}'}}], 'token_count': {'input_tokens': 994.0, 'output_tokens': 90.0}}, response_metadata={'id': 'afcba0ba-afb5-44d2-b85b-b8de470fd61f', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'First, I will find out the conversion fac

In [36]:
res2 = binded_llm.invoke(messages)
res2

AIMessage(content='The conversion factor from EUR to INR is 96.993. Now, I will multiply this conversion factor by 86 to convert 86 EUR into INR.', additional_kwargs={'id': 'bf8976ca-7ca0-4ab5-a3e6-ce972fed98f2', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'The conversion factor from EUR to INR is 96.993. Now, I will multiply this conversion factor by 86 to convert 86 EUR into INR.', 'tool_calls': [{'id': 'convertCurrency_bkycqdfwkn1e', 'type': 'function', 'function': {'name': 'convertCurrency', 'arguments': '{"amount":86,"conversion_factor":96.993}'}}], 'token_count': {'input_tokens': 1304.0, 'output_tokens': 92.0}}, response_metadata={'id': 'bf8976ca-7ca0-4ab5-a3e6-ce972fed98f2', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'The conversion factor from EUR to INR is 96.993. Now, I will multiply this conversion factor by 86 to convert 86 EUR into INR.', 'tool_calls': [{'id': 'convertCurrency_bkycqdfwkn1e', 'type': 'function', 'function': {'name': 'convertCurrency', 'arguments': '{"amount"

In [37]:
# conversion_rate = float(getconversionrateResponse.content)
# res2.tool_calls[0]['args']['conversion_factor'] = conversion_rate
multiplierResponse = convertCurrency.invoke(res2.tool_calls[0])
multiplierResponse

ToolMessage(content='8341.398', name='convertCurrency', tool_call_id='convertCurrency_bkycqdfwkn1e')

In [38]:
messages.append(res2)
messages.append(multiplierResponse)
messages

[HumanMessage(content='What is the conversion factor from eur to inr, and based on that can you convert 86 eur to INR?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='First, I will find out the conversion factor from EUR to INR. Then, I will multiply this conversion factor by 86 to convert 86 EUR into INR.', additional_kwargs={'id': 'afcba0ba-afb5-44d2-b85b-b8de470fd61f', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'First, I will find out the conversion factor from EUR to INR. Then, I will multiply this conversion factor by 86 to convert 86 EUR into INR.', 'tool_calls': [{'id': 'getCurrencyConversionFactor_272vhd9hntva', 'type': 'function', 'function': {'name': 'getCurrencyConversionFactor', 'arguments': '{"base_currency":"EUR","target_currency":"INR"}'}}], 'token_count': {'input_tokens': 994.0, 'output_tokens': 90.0}}, response_metadata={'id': 'afcba0ba-afb5-44d2-b85b-b8de470fd61f', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'First, I will find out the conversion fac

In [39]:
final_response = binded_llm.invoke(messages)

In [40]:
final_response

AIMessage(content='The conversion factor from EUR to INR is 96.993. 86 EUR is equivalent to approximately **8341.398 INR**.', additional_kwargs={'id': '59c67149-a484-43cf-9851-e8543c49c622', 'finish_reason': 'COMPLETE', 'content': 'The conversion factor from EUR to INR is 96.993. 86 EUR is equivalent to approximately **8341.398 INR**.', 'citations': [Citation(start=41, end=47, text='96.993', sources=[ToolSource(type='tool', id='getCurrencyConversionFactor_272vhd9hntva:0', tool_output={'output': '{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1745971201, "time_last_update_utc": "Wed, 30 Apr 2025 00:00:01 +0000", "time_next_update_unix": 1746057601, "time_next_update_utc": "Thu, 01 May 2025 00:00:01 +0000", "base_code": "EUR", "target_code": "INR", "conversion_rate": 96.993}'})], type='TEXT_CONTENT'), Citation(start=88, end=101, text='*8341.398 INR', sources=[ToolSource(typ

In [41]:
import json
for tool_call in res1.tool_calls:
    # get the conversion factor
    if tool_call['name'] == 'getCurrencyConversionFactor':
        conversion_factor_response = getCurrencyConversionFactor.invoke(tool_call)
        # fetch conversion rate
        conversion_rate = json.loads(conversion_factor_response.content)
        messages.append(conversion_factor_response)
        
    else:
        # add conversion factor to the tool call
        tool_call[0]['args']['conversion_factor'] = conversion_rate
        # get the conversion amount
        conversion_amount_response = convertCurrency.invoke(tool_call)
        messages.append(conversion_amount_response)